# Advent of Code 2022

## Day 23: Unstable Diffusion

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

Not too hard today. The class which does most of the work is `State`, which has a `step` method which moves the elves one round. A lot of this class is rendering code for debugging in the string method. For part 1, I just go through a wile loop until the number of elves moves is 0.

### Imports

In [ ]:
from collections import namedtuple
import tqdm.notebook as tqdm

### Position/Movement Stuff

In [ ]:
# noinspection PyTypeChecker
Point = namedtuple('Point', ['x', 'y'])

# noinspection PyTypeChecker
Delta = namedtuple('Delta', ['dx', 'dy'])

Point.__add__ = lambda self, delta: Point(x=self.x+delta.dx, y=self.y+delta.dy)


class Consideration:

    __slots__ = ['name', 'icon', 'deltas', 'relative']

    def __init__(self, name: str, icon: str, relative: Delta, deltas: tuple[Delta, Delta, Delta]):
        self.name = name
        self.icon = icon
        assert len(deltas) == 3
        self.deltas = deltas
        self.relative = relative

    def is_valid(self, location: Point, locations: set[Point]):
        for delta in self.deltas:
            if location + delta in locations:
                return False
        return True

    def __str__(self):
        return self.icon

    def __repr__(self):
        return self.name

NORTH = Consideration('NORTH', '⬆️', Delta(dx=0, dy=-1), (Delta(dx=-1, dy=-1), Delta(dx=0, dy=-1), Delta(dx=1, dy=-1)))
SOUTH = Consideration('SOUTH', '⬇️', Delta(dx=0, dy=1), (Delta(dx=-1, dy=1), Delta(dx=0, dy=1), Delta(dx=1, dy=1)))
WEST = Consideration('WEST', '⬅️', Delta(dx=-1, dy=0), (Delta(dx=-1, dy=-1), Delta(dx=-1, dy=0), Delta(dx=-1, dy=1)))
EAST = Consideration('EAST', '➡️', Delta(dx=1, dy=0), (Delta(dx=1, dy=-1), Delta(dx=1, dy=0), Delta(dx=1, dy=1)))

SEQUENCE = NORTH, SOUTH, WEST, EAST

ALL_8_SPACES = (Delta(dx=-1, dy=-1), Delta(dx=0, dy=-1), Delta(dx=1, dy=-1),
                Delta(dx=-1, dy=0),                      Delta(dx=1, dy=0),
                Delta(dx=-1, dy=1), Delta(dx=0, dy=1), Delta(dx=1, dy=1))

### The Elf Class

In [ ]:
class Elf:

    __slots__ = ['hash_proxy', 'location']
    next_hash_proxy = 0

    def __init__(self, location: Point):
        self.hash_proxy = Elf.next_hash_proxy
        Elf.next_hash_proxy += 1
        self.location = location

    def has_space(self, locations: set[Point]):
        for rel in ALL_8_SPACES:
            if self.location + rel in locations:
                return False
        return True

    def __str__(self):
        return '🧝'

    def __repr__(self):
        return f'Elf(hash={self.hash_proxy}, location={repr(self.location)})'

    def __hash__(self):
        return hash(self.hash_proxy)

    def __eq__(self, other: 'Elf') -> bool:
        if type(other) != Elf:
            return False
        return self.hash_proxy == other.hash_proxy

    def __ne__(self, other: 'Elf') -> bool:
        if type(other) != Elf:
            return True
        return self.hash_proxy != other.hash_proxy

Elf.next_hash_proxy = 0

### The State Class

In [ ]:
class State:

    __slots__ = ['elves', 'plan', 'elf_count', 'last_elves_moved', 'rounds']

    def __init__(self, raw: list[str]):
        assert type(raw) == list
        self.elf_count = 0
        self.last_elves_moved = None
        height = len(raw)
        if not height:
            raise ValueError('empty input')
        width = None
        for line in raw:
            assert type(line) == str
            if width is not None and len(line) != width:
                raise ValueError('non-rectangular input')
            width = len(line)
        if width is None or width < 1:
            raise ValueError('zero width')
        self.elves = {}
        for y, line in enumerate(raw):
            for x, char in enumerate(line):
                if char not in {'.', '#'}:
                    raise ValueError(f'unknown symbol: {repr(char)}')
                if char == '#':
                    self.elf_count += 1
                    point = Point(x=x, y=y)
                    assert point not in self.elves
                    self.elves[point] = Elf(point)
        self.plan = SEQUENCE
        self.rounds = 0

    def bounding_box(self) -> tuple[tuple[int, int], tuple[int, int]]:
        x_min, x_max = None, None
        y_min, y_max = None, None
        for point in self.elves:
            if x_min is None or point.x < x_min:
                x_min = point.x
            if x_max is None or point.x > x_max:
                x_max = point.x
            if y_min is None or point.y < y_min:
                y_min = point.y
            if y_max is None or point.y > y_max:
                y_max = point.y
        assert x_min is not None
        assert x_max is not None
        assert y_min is not None
        assert y_max is not None
        return (x_min, x_max), (y_min, y_max)

    def bounding_box_size(self):
        x_range, y_range = self.bounding_box()
        x_min, x_max = x_range
        y_min, y_max = y_range
        return (x_max + 1 - x_min) * (y_max + 1 - y_min)

    def step(self):

        # phase 1
        next_location_to_elf_list: dict[Point, list[Elf]] = {}
        elf_to_next_location: dict[Elf, Point] = {}
        elf_to_consideration: dict[Elf, Consideration] = {}
        for point, elf in self.elves.items():
            if elf.has_space(self.elves.keys()):
                next_location_to_elf_list[point] = [elf]
                elf_to_next_location[elf] = point
        for con in self.plan:
            for point, elf in self.elves.items():
                if elf not in elf_to_next_location and con.is_valid(point, self.elves.keys()):
                    next_location = point + con.relative
                    elf_to_next_location[elf] = next_location
                    elf_to_consideration[elf] = con
                    if next_location in next_location_to_elf_list:
                        next_location_to_elf_list[next_location].append(elf)
                    else:
                        next_location_to_elf_list[next_location] = [elf]

        #print(self.to_str(elf_to_consideration))

        #print(self.to_str(elf_to_consideration, {key: len(value) for key, value in next_location_to_elf_list.items()}))

        # phase 2
        elves_moved = 0
        for next_location, elves in next_location_to_elf_list.items():
            assert len(elves) > 0
            if len(elves) == 1:
                elf ,= elves
                old_location = elf.location
                if old_location != next_location:
                    elf.location = next_location
                    del self.elves[old_location]
                    assert next_location not in self.elves
                    self.elves[next_location] = elf
                    elves_moved += 1
        self.last_elves_moved = elves_moved

        # cycle plan
        first, rest = self.plan[0:1], self.plan[1:]
        self.plan = rest + first
        new_value = self.rounds + 1
        self.rounds = new_value
        assert self.rounds == new_value

    def to_str(self, elf_to_consideration: dict[Elf, Consideration] = None, next_location_to_elf_count: dict[Point, int] = None):
        if elf_to_consideration is None:
            elf_to_consideration = {}
        rv = ''
        elf_count = 0
        x_range, y_range = self.bounding_box()
        x_min, x_max = x_range
        y_min, y_max = y_range
        points = 0
        for y in range(y_min, y_max + 1):
            for x in range(x_min, x_max + 1):
                point = Point(x=x, y=y)
                if point in self.elves:
                    assert (self.elves[point].location == point), f'elf mapped to location {point} records own location as {self.elves[point].location}'
                    elf = self.elves[point]
                    if elf in elf_to_consideration:
                        if next_location_to_elf_count is not None:
                            next_point = point + elf_to_consideration[elf].relative
                            assert next_point in next_location_to_elf_count
                            assert next_location_to_elf_count[next_point] > 0
                            if next_location_to_elf_count[next_point] == 1:
                                rv += elf_to_consideration[elf].icon
                            else:
                                rv += '❌'
                        else:
                            rv += elf_to_consideration[elf].icon
                    else:
                        rv += str(self.elves[point])
                    elf_count += 1
                else:
                    rv += '⬛'
                points += 1
            rv += '\n'
        if elf_count != self.elf_count or elf_count != len(self.elves):
            raise ValueError(f'elf count mismatch: __str__.{elf_count = }, {self.elf_count = }, {len(self.elves) = }')
        size = self.bounding_box_size()
        assert size == points
        rv += f'elves: {elf_count}\n'
        rv += f'elves moved: {self.last_elves_moved}\n'
        rv += f'bounding box size: {size}\n'
        rv += f'empty squares: {size - elf_count}\n'
        rv += f'plan: ' + ', '.join([str(plan) for plan in self.plan])
        rv += f'\nrounds: {self.rounds}\n'
        return rv + '\n'

    def __str__(self):
        return self.to_str()

    def __repr__(self):
        return str(self)

    @staticmethod
    def read(filename: str) -> 'State':
        with open(filename) as file:
            raw = []
            for line in file:
                raw.append(line.strip())
            return State(raw)

### Part 1

In [ ]:
def main():
    state = State.read(INPUT_FILE)
    for _ in range(NUM_ROUNDS):
        state.step()
    print(f'The number of empty squares is {state.bounding_box_size() - state.elf_count}')

In [ ]:
NUM_ROUNDS = 10
INPUT_FILE = 'data/input23.txt'

if __name__ == '__main__':
    main()

### Part 2

In [ ]:
def main():
    state = State.read(INPUT_FILE)
    with tqdm.tqdm() as pbar:
        while state.last_elves_moved != 0:
            state.step()
            pbar.desc = str(state.last_elves_moved) + ' elves moved in last round'
            pbar.update(1)
    print(f'The number of rounds is {state.rounds}')

In [ ]:
INPUT_FILE = 'data/input23.txt'

if __name__ == '__main__':
    main()